<a href="https://colab.research.google.com/github/jfzhang726/finetune-reranker/blob/main/reranker_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<b>Traning needs L4 because T4 GPU memory is not enough.

<b>Evalueation uses T4. It is weird that evaluation will be stuck after inference embedding on T4.

## 0. Preparation

### 0.1 Mount google drive.

Finetuned models will be saved to google drive.

In [ ]:
from google.colab import drive
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Mount your Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


### 0.2 Install packages.


Troubleshoot:
- If there is error "google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible", execute the cell again and the error may go away.


In [ ]:
%%bash
# git clone https://github.com/FlagOpen/FlagEmbedding.git
# cd FlagEmbedding
# pip install -e .

pip install -U FlagEmbedding

pip install LM_Cocktail

pip install accelerate>=0.20.1

pip install transformers>=4.33.0

pip install datasets

pip install faiss-gpu



In [ ]:
!pip install accelerate

In [ ]:
import os
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import shutil

In [ ]:
# have a look at BGE reranker


tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-reranker-base')
model = AutoModelForSequenceClassification.from_pretrained('BAAI/bge-reranker-base')
model.eval()

pairs = [['what is panda?', 'hi'], ['what is panda?', 'The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.']]
with torch.no_grad():
    inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
    scores = model(**inputs, return_dict=True).logits.view(-1, ).float()
    print(scores)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tensor([-8.1544,  6.1821])


### 0.3 Specify dirs where models where be saved



In [ ]:
# Specify paths for saving models and data

# give a name to the finedtuned reranker that will be generated
model_variation = "scidocs_3k_all_pos_15epoch"

# the dir in colab instance
output_dir = f"models/{model_variation}"

# dir for saving finetuned reranker
output_dir_finetuned = os.path.join(output_dir, "finetuned")
# dir for saving the model merged from finetuned model and original model
output_dir_merged = os.path.join(output_dir, "merged")

# Create a directory in Google Drive to save the model files
drive_output_dir = f"/content/drive/My Drive/Colab Notebooks/reranker/models/{model_variation}/"
# dir for saving finetuned reranker
drive_output_dir_finetuned = os.path.join(drive_output_dir, "finetuned")
# dir for saving the model merged from finetuned model and original model
drive_output_dir_merged = os.path.join(drive_output_dir, "merged")



## 1. Finetune
Use mteb/scidoc-reranking dataset to finetune BGE reranker BAAI/bge-reranker-base. Validation split of mteb/scidoc-reranking is used as training data, and test split is used for evaluation.
- Base reranker model
  - BAAI/bge-reranker-base
  - https://huggingface.co/BAAI/bge-reranker-base
- Finetune dataset
  - Scidoc-reranking dataset
  - https://huggingface.co/datasets/mteb/scidocs-reranking


reference: https://github.com/FlagOpen/FlagEmbedding/tree/master/examples/finetune

### 1.1 Prepare data

In [ ]:
train_data_file = "./scidocs/validation.jsonl"


# load dataset mteb/scidocs-reranking from huggingface
from datasets import load_dataset
dataset = load_dataset("mteb/scidocs-reranking")

# create a new dataset from the loaded dataset where the list of "positive" contains one positive sample randomly selected from the original "positive" list
import random
from datasets import DatasetDict, Dataset

def rename_keys(example):
  example['pos'] = example['positive']
  example.pop('positive')
  example['neg'] = example['negative']
  example.pop('negative')
  return example

def select_random_positive(example):
  example[f'pos'] = [random.choice(example['positive'])]
  example[f'neg'] = example.pop('negative')
  example.pop('positive')
  return example

def select_first_positive(example):
  example[f'pos'] = [example['positive'][0]]
  example[f'neg'] = example.pop('negative')
  example.pop('positive')
  return example


new_dataset = DatasetDict({
    split_name: dataset[split_name].map(rename_keys, num_proc=4) for split_name in dataset})

new_dataset['validation'].to_json(train_data_file, orient="records", lines=True)
# new_dataset['test'].to_json("scidocs/test.jsonl", orient="records", lines=True)
# small_dataset = new_dataset['validation'].select(range(256))
# small_dataset.to_json("scidocs/small_validation.jsonl", orient="records", lines=True)

negative_numbers = [len(negative_list) for negative_list in new_dataset['validation']['neg']]
min_negative_number = min(negative_numbers)
max_negative_number = max(negative_numbers)
print(min_negative_number, max_negative_number)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating validation split:   0%|          | 0/3981 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3978 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3981 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3978 [00:00<?, ? examples/s]

Creating json from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

22 49


### 1.2 Training

In [ ]:
#


# Define the paths and other parameters

# model_variation = "scidocs_3k_all_pos_20epoch"

# output_dir = f"models/{model_variation}"

# # Create a directory in Google Drive to save the model files
# drive_output_dir = f"/content/drive/My Drive/Colab Notebooks/reranker/models/{model_variation}/"

# output_dir_finetuned = os.path.join(output_dir, "finetuned")
# output_dir_merged = os.path.join(output_dir, "merged")

# drive_output_dir_finetuned = os.path.join(drive_output_dir, "finetuned")
# drive_output_dir_merged = os.path.join(drive_output_dir, "merged")

model_name_or_path = "BAAI/bge-reranker-base"
learning_rate = 6e-5
num_train_epochs = 15
per_device_train_batch_size = 8 #16
gradient_accumulation_steps = 8 #4
train_group_size = min_negative_number + 1
max_len = 512
weight_decay = 0.01
logging_steps = 10

# Construct the command using string formatting
command = f"""
torchrun --nproc_per_node 1 \
-m FlagEmbedding.reranker.run \
--output_dir {output_dir_finetuned} \
--model_name_or_path {model_name_or_path} \
--train_data {train_data_file} \
--learning_rate {learning_rate} \
--fp16 \
--num_train_epochs {num_train_epochs} \
--per_device_train_batch_size {per_device_train_batch_size} \
--gradient_accumulation_steps {gradient_accumulation_steps} \
--dataloader_drop_last True \
--train_group_size {train_group_size} \
--max_len {max_len} \
--weight_decay {weight_decay} \
--logging_steps {logging_steps}
"""



In [ ]:
# Execute the command to finetune reranker
!{command}

2024-06-15 07:25:32.594105: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-15 07:25:32.648264: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-15 07:25:32.648314: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-15 07:25:32.650284: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-15 07:25:32.658796: I tensorflow/core/platform/cpu_feature_guar

<b> Save model to google drive </b>

In [ ]:

import shutil
import os





# Copy the entire output directory to Google Drive
shutil.copytree(output_dir_finetuned, drive_output_dir_finetuned)

print(f"Model files are saved to {drive_output_dir_finetuned}")

Model files are saved to /content/drive/My Drive/Colab Notebooks/reranker/models/scidocs_3k_all_pos_15epoch/finetuned


## 1.3 merged finetuned model with original model

<b> Load finetuned from google drive. </b>

In [ ]:

import torch

# Check if a GPU is available and move the model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(drive_output_dir_finetuned)
model = AutoModelForSequenceClassification.from_pretrained(drive_output_dir_finetuned)
model.to(device)

# Use the model as needed
model.eval()

pairs = [['what is panda?', 'hi'], ['what is panda?', 'The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.']]
with torch.no_grad():
    inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512).to(device)
    scores = model(**inputs, return_dict=True).logits.view(-1, ).float()
    print(scores)

tensor([-5.8636,  9.7072], device='cuda:0')


<b> Merge finetuned model with original model </b>

In [ ]:
!pip install accelerate

In [ ]:
from LM_Cocktail import mix_models, mix_models_with_data

# Mix fine-tuned model and base model; then save it to output_path: ./mixed_model_1
model = mix_models(
    model_names_or_paths=["BAAI/bge-reranker-base", output_dir_finetuned],
    model_type='reranker',
    weights=[0.5, 0.5],  # you can change the weights to get a better trade-off.
    output_path=output_dir_merged)

loading BAAI/bge-reranker-base -----------------


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


loading models/scidocs_3k_all_pos_15epoch/finetuned -----------------
***weight for each model***: 
BAAI/bge-reranker-base 0.5
models/scidocs_3k_all_pos_15epoch/finetuned 0.5
Saving the new model to models/scidocs_3k_all_pos_15epoch/merged


<b> Save merged model to Google Drive</b>

In [ ]:


# Copy the entire output directory to Google Drive
shutil.copytree(output_dir_merged, drive_output_dir_merged)

print(f"Model files are saved to {drive_output_dir_merged}")

Model files are saved to /content/drive/My Drive/Colab Notebooks/reranker/models/scidocs_3k_all_pos_15epoch/merged


<b> Confirm the merged model on Google Drive is correct. </b>

In [ ]:
import torch

# Check if a GPU is available and move the model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(drive_output_dir_merged)
model = AutoModelForSequenceClassification.from_pretrained(drive_output_dir_merged)
model.to(device)

# Use the model as needed
model.eval()

pairs = [['what is panda?', 'hi'], ['what is panda?', 'The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.']]
with torch.no_grad():
    inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512).to(device)
    # inputs = {key: value.to(device) for key, value in inputs.items()}
    scores = model(**inputs, return_dict=True).logits.view(-1, ).float()
    print(scores)

tensor([-7.0765,  8.1053], device='cuda:0')


## 1.4 Evaluation (MRR and Hit Rate)
- NOTE: change runtime type to T4
  - It is weird that evaluation will be stuck after "Inference Embedding".
- Use test split of mteb/scidocs-reranking evaluate finetuned models (not merged and merged)
- Adapt evaluation script at https://github.com/FlagOpen/FlagEmbedding/blob/master/FlagEmbedding/baai_general_embedding/finetune/eval_msmarco.py by adding evaluation of reranking

### 1.4.1 prepare dataset



reference: https://github.com/FlagOpen/FlagEmbedding/tree/master/examples/finetune#62-your-dataset

In [ ]:
eval_corpus_file = "scidocs/eval_corpus.jsonl"
eval_queries_file = "scidocs/eval_queries.jsonl"


# load dataset mteb/scidocs-reranking from huggingface
from datasets import load_dataset, Dataset
import itertools
dataset = load_dataset("mteb/scidocs-reranking")
test_dataset = dataset['test']
eval_corpus = test_dataset['positive'] + test_dataset['negative']
eval_corpus = list(itertools.chain(*eval_corpus))
eval_corpus = [{"content": text} for text in eval_corpus]
print(f"eval_corpus {len(eval_corpus)}")
eval_corpus = Dataset.from_list(eval_corpus)
eval_corpus.to_json(eval_corpus_file, orient="records", lines=True)

eval_queries = [{'query': example['query'], 'positive': example['positive']} for example in test_dataset]
print(f"eval_queries {len(eval_queries)}")
eval_queries = Dataset.from_list(eval_queries)
eval_queries.to_json(eval_queries_file, orient="records", lines=True)


eval_corpus 118600


Creating json from Arrow format:   0%|          | 0/119 [00:00<?, ?ba/s]

eval_queries 3978


Creating json from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

1795563

### 1.4.2 evaluate

Compare the results of
 - BGE retrieval
 - BGE retrieval + BGE reranker
 - BGE retrieval + finetuned reranker
 - BGE retrieval + finetuned reranker merged with GGE reranker

In [ ]:
# code adapted from script https://github.com/FlagOpen/FlagEmbedding/blob/master/FlagEmbedding/baai_general_embedding/finetune/eval_msmarco.py
# - add reranking
# - add saving results

import faiss
import torch
import logging
import datasets
import numpy as np
from tqdm import tqdm
from typing import Optional
from dataclasses import dataclass, field
from transformers import HfArgumentParser
from FlagEmbedding import FlagModel
from FlagEmbedding import FlagReranker
import json
import os
from collections import OrderedDict
from pprint import pprint



logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

@dataclass
class Args:
    encoder: str = field(
        default="BAAI/bge-base-en-v1.5",
        metadata={'help': 'The encoder name or path.'}
    )
    fp16: bool = field(
        default=False,
        metadata={'help': 'Use fp16 in inference?'}
    )
    add_instruction: bool = field(
        default=False,
        metadata={'help': 'Add query-side instruction?'}
    )

    corpus_data: str = field(
        default="namespace-Pt/msmarco",
        metadata={'help': 'candidate passages'}
    )
    query_data: str = field(
        default="namespace-Pt/msmarco-corpus",
        metadata={'help': 'queries and their positive passages for evaluation'}
    )

    max_query_length: int = field(
        default=32,
        metadata={'help': 'Max query length.'}
    )
    max_passage_length: int = field(
        default=128,
        metadata={'help': 'Max passage length.'}
    )
    batch_size: int = field(
        default=256,
        metadata={'help': 'Inference batch size.'}
    )
    index_factory: str = field(
        default="Flat",
        metadata={'help': 'Faiss index factory.'}
    )
    k: int = field(
        default=100,
        metadata={'help': 'How many neighbors to retrieve?'}
    )

    save_embedding: bool = field(
        default=False,
        metadata={'help': 'Save embeddings in memmap at save_dir?'}
    )
    load_embedding: bool = field(
        default=False,
        metadata={'help': 'Load embeddings from save_dir?'}
    )
    save_path: str = field(
        default="embeddings.memmap",
        metadata={'help': 'Path to save embeddings.'}
    )
    save_metrics: bool = field(
        default=False,
        metadata={'help': 'Save metrics?'}
    )
    save_metrics_path: str = field(
        default="./",
        metadata={'help': 'Path to save metrics.'}
    )
    save_retrieval_results: bool = field(
        default=False,
        metadata={'help': 'Save retrieval results?'}
    )
    save_retrieval_results_path: str = field(
        default="./",
        metadata={'help': 'Path to save retrieval results.'}
    )
    save_reranking_results: bool = field(
        default=False,
        metadata={'help': 'Save reranking results?'}
    )
    save_reranking_results_path: str = field(
        default="./",
        metadata={'help': 'Path to save reranking results.'}
    )



def index(model: FlagModel, corpus: datasets.Dataset, batch_size: int = 256, max_length: int=512, index_factory: str = "Flat", save_path: str = None, save_embedding: bool = False, load_embedding: bool = False):
    """
    1. Encode the entire corpus into dense embeddings;
    2. Create faiss index;
    3. Optionally save embeddings.
    """
    if load_embedding:
        test = model.encode("test")
        dtype = test.dtype
        dim = len(test)

        corpus_embeddings = np.memmap(
            save_path,
            mode="r",
            dtype=dtype
        ).reshape(-1, dim)

    else:
        corpus_embeddings = model.encode_corpus(corpus["content"], batch_size=batch_size, max_length=max_length)
        dim = corpus_embeddings.shape[-1]

        if save_embedding:
            logger.info(f"saving embeddings at {save_path}...")
            memmap = np.memmap(
                save_path,
                shape=corpus_embeddings.shape,
                mode="w+",
                dtype=corpus_embeddings.dtype
            )

            length = corpus_embeddings.shape[0]
            # add in batch
            save_batch_size = 10000
            if length > save_batch_size:
                for i in tqdm(range(0, length, save_batch_size), leave=False, desc="Saving Embeddings"):
                    j = min(i + save_batch_size, length)
                    memmap[i: j] = corpus_embeddings[i: j]
            else:
                memmap[:] = corpus_embeddings

    # create faiss index
    faiss_index = faiss.index_factory(dim, index_factory, faiss.METRIC_INNER_PRODUCT)

    if model.device == torch.device("cuda"):
        # co = faiss.GpuClonerOptions()
        co = faiss.GpuMultipleClonerOptions()
        co.useFloat16 = True
        # faiss_index = faiss.index_cpu_to_gpu(faiss.StandardGpuResources(), 0, faiss_index, co)
        faiss_index = faiss.index_cpu_to_all_gpus(faiss_index, co)

    # NOTE: faiss only accepts float32
    logger.info("Adding embeddings...")
    corpus_embeddings = corpus_embeddings.astype(np.float32)
    faiss_index.train(corpus_embeddings)
    faiss_index.add(corpus_embeddings)
    return faiss_index


def search(model: FlagModel, queries: datasets, faiss_index: faiss.Index, k:int = 100, batch_size: int = 256, max_length: int=512):
    """
    1. Encode queries into dense embeddings;
    2. Search through faiss index
    """
    query_embeddings = model.encode_queries(queries["query"], batch_size=batch_size, max_length=max_length)
    query_size = len(query_embeddings)

    all_scores = []
    all_indices = []

    for i in tqdm(range(0, query_size, batch_size), desc="Searching"):
        j = min(i + batch_size, query_size)
        query_embedding = query_embeddings[i: j]
        score, indice = faiss_index.search(query_embedding.astype(np.float32), k=k)
        all_scores.append(score)
        all_indices.append(indice)

    all_scores = np.concatenate(all_scores, axis=0)
    all_indices = np.concatenate(all_indices, axis=0)
    return all_scores, all_indices


def rerank(model_name_or_path, queries, retrieval_results):
    reranking_pairs = []
    for i in range(len(queries)):
      pairs_for_query = []
      for j in range(len(retrieval_results[i])):
        pairs_for_query.append((queries[i]['query'], retrieval_results[i][j]))
      reranking_pairs.append(pairs_for_query)

    reranking_scores = []



    # Check if a GPU is available and move the model to GPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
    model = AutoModelForSequenceClassification.from_pretrained(model_name_or_path)
    model.to(device)

    # Use the model as needed
    model.eval()
    with torch.no_grad():
      for i in tqdm(range(0, len(reranking_pairs)), desc="reranking"):
        pairs = reranking_pairs[i]
        inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
        scores = model(**inputs, return_dict=True).logits.view(-1, ).float()
        # print(scores)
        reranking_scores.append(scores.tolist())

    retrieval_results_sorted = []
    for i in range(len(reranking_pairs)):
      combined = list(zip(reranking_pairs[i], reranking_scores[i]))
      sorted_combined = sorted(combined, key=lambda x: x[1], reverse=True)
      retrieval_results_sorted.append(sorted_combined)

    reranking_results = []
    for i in range(len(retrieval_results_sorted)):
      reranking_results.append([x[0][1] for x in retrieval_results_sorted[i]])
    return reranking_results, retrieval_results_sorted


def rerank_flag_model(model_name_or_path, queries, retrieval_results):
    reranking_pairs = []
    for i in range(len(queries)):
      pairs_for_query = []
      for j in range(len(retrieval_results[i])):
        pairs_for_query.append((queries[i]['query'], retrieval_results[i][j]))
      reranking_pairs.append(pairs_for_query)

    reranking_scores = []

    reranker = FlagReranker(model_name_or_path, use_fp16=True) #use fp16 can speed up computing
    for i in tqdm(range(0, len(reranking_pairs)), desc="reranking"):
        pairs = reranking_pairs[i]
        scores = reranker.compute_score(pairs)
        reranking_scores.append(scores)
        retrieval_results_sorted = []


    for i in range(len(reranking_pairs)):
      combined = list(zip(reranking_pairs[i], reranking_scores[i]))
      sorted_combined = sorted(combined, key=lambda x: x[1], reverse=True)
      retrieval_results_sorted.append(sorted_combined)

    reranking_results = []
    for i in range(len(retrieval_results_sorted)):
      reranking_results.append([x[0][1] for x in retrieval_results_sorted[i]])
    return reranking_results, retrieval_results_sorted


def evaluate(preds, labels, cutoffs=[1,10,100]):
    """
    Evaluate MRR and Recall at cutoffs.
    """
    metrics = {}

    # MRR
    mrrs = np.zeros(len(cutoffs))
    for pred, label in zip(preds, labels):
        jump = False
        for i, x in enumerate(pred, 1):
            if x in label:
                for k, cutoff in enumerate(cutoffs):
                    if i <= cutoff:
                        mrrs[k] += 1 / i
                jump = True
            if jump:
                break
    mrrs /= len(preds)
    for i, cutoff in enumerate(cutoffs):
        mrr = mrrs[i]
        metrics[f"MRR@{cutoff}"] = mrr

    # Recall
    recalls = np.zeros(len(cutoffs))
    for pred, label in zip(preds, labels):
        for k, cutoff in enumerate(cutoffs):
            recall = np.intersect1d(label, pred[:cutoff])
            recalls[k] += len(recall) / len(label)
    recalls /= len(preds)
    for i, cutoff in enumerate(cutoffs):
        recall = recalls[i]
        metrics[f"Recall@{cutoff}"] = recall

    return metrics


def main(args):
    # parser = HfArgumentParser([Args])
    # args: Args = parser.parse_args_into_dataclasses()[0]

    if args.query_data == 'namespace-Pt/msmarco-corpus':
        assert args.corpus_data == 'namespace-Pt/msmarco'
        eval_data = datasets.load_dataset("namespace-Pt/msmarco", split="dev")
        corpus = datasets.load_dataset("namespace-Pt/msmarco-corpus", split="train")
    else:
        eval_data = datasets.load_dataset('json', data_files=args.query_data, split='train')
        corpus = datasets.load_dataset('json', data_files=args.corpus_data, split='train')

    all_metrics = OrderedDict()
    print("loading embedding model")
    model = FlagModel(
        args.encoder,
        query_instruction_for_retrieval="Represent this sentence for searching relevant passages: " if args.add_instruction else None,
        use_fp16=args.fp16
    )
    print("building index")
    faiss_index = index(
        model=model,
        corpus=corpus,
        batch_size=args.batch_size,
        max_length=args.max_passage_length,
        index_factory=args.index_factory,
        save_path=args.save_path,
        save_embedding=args.save_embedding,
        load_embedding=args.load_embedding
    )
    print("retrieval")
    scores, indices = search(
        model=model,
        queries=eval_data,
        faiss_index=faiss_index,
        k=args.k,
        batch_size=args.batch_size,
        max_length=args.max_query_length
    )

    retrieval_results = []
    for indice in indices:
        # filter invalid indices
        indice = indice[indice != -1].tolist()
        retrieval_results.append(corpus[indice]["content"])

    ground_truths = []
    for sample in eval_data:
        ground_truths.append(sample["positive"])


    metrics = evaluate(retrieval_results, ground_truths)
    all_metrics["retrieval"] = metrics
    print(metrics)
    if args.save_retrieval_results:
      with open(os.path.join(args.save_retrieval_results_path, "retrieval_results.json"), "w") as f:
        json.dump(retrieval_results, f, indent=4)


    # Load reranking tokenizer and model
    print("BGE reranking")
    # tokenizer_reranking = AutoTokenizer.from_pretrained('BAAI/bge-reranker-base')
    # model_reranking = AutoModelForSequenceClassification.from_pretrained('BAAI/bge-reranker-base')
    reranking_results, retrieval_results_sorted = rerank_flag_model(model_name_or_path='BAAI/bge-reranker-base',
                               queries=eval_data,
                               retrieval_results=retrieval_results)

    metrics = evaluate(reranking_results, ground_truths)
    all_metrics["bge_reranking"] = metrics

    print(metrics)
    if args.save_reranking_results:
      with open(os.path.join(args.save_reranking_results_path, "bge_reranking_results.json"), "w") as f:
        json.dump(reranking_results, f, indent=4)
      with open(os.path.join(args.save_reranking_results_path, "bge_reranking_results_sorted.json"), "w") as f:
        json.dump(retrieval_results_sorted, f, indent=4)


    print("finetuned reranking")
    # tokenizer_reranking = AutoTokenizer.from_pretrained(drive_output_dir_finetuned)
    # model_reranking = AutoModelForSequenceClassification.from_pretrained(drive_output_dir_finetuned)
    reranking_results, retrieval_results_sorted = rerank_flag_model(model_name_or_path=drive_output_dir_finetuned,
                               queries=eval_data,
                               retrieval_results=retrieval_results)

    metrics = evaluate(reranking_results, ground_truths)
    all_metrics["finetuned_reranking"] = metrics
    print(metrics)
    if args.save_reranking_results:
      with open(os.path.join(args.save_reranking_results_path, "finetuned_reranking_results.json"), "w") as f:
        json.dump(reranking_results, f, indent=4)
      with open(os.path.join(args.save_reranking_results_path, "finetuned_reranking_results_sorted.json"), "w") as f:
        json.dump(retrieval_results_sorted, f, indent=4)


    print("merged reranking")
    # Load reranking tokenizer and model
    # tokenizer_reranking = AutoTokenizer.from_pretrained(drive_output_dir_merged)
    # model_reranking = AutoModelForSequenceClassification.from_pretrained(drive_output_dir_merged)
    reranking_results, retrieval_results_sorted = rerank_flag_model(model_name_or_path=drive_output_dir_merged,
                               queries=eval_data,
                               retrieval_results=retrieval_results)

    metrics = evaluate(reranking_results, ground_truths)
    all_metrics["merged_reranking"] = metrics
    print(metrics)
    if args.save_reranking_results:
      with open(os.path.join(args.save_reranking_results_path, "merged_reranking_results.json"), "w") as f:
        json.dump(reranking_results, f, indent=4)
      with open(os.path.join(args.save_reranking_results_path, "merged_reranking_results_sorted.json"), "w") as f:
        json.dump(retrieval_results_sorted, f, indent=4)


    if args.save_metrics:
      with open(os.path.join(args.save_metrics_path, "metrics.json"), "w") as f:
        json.dump(all_metrics, f, indent=4)
    pprint(all_metrics)

# if __name__ == "__main__":
#     main()



In [ ]:
os.makedirs("./scidocs/eval", exist_ok=True)

args = Args(
    encoder='BAAI/bge-base-en-v1.5',
    fp16=True,
    add_instruction=False,
    k=100,
    corpus_data=eval_corpus_file,
    query_data=eval_queries_file,
    save_metrics=True,
    save_metrics_path="./scidocs/eval/",
    save_retrieval_results=True,
    save_retrieval_results_path="./scidocs/eval/",
    save_reranking_results=True,
    save_reranking_results_path="./scidocs/eval/",
)

main(args)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

loading embedding model


tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

building index


Inference Embeddings: 100%|██████████| 464/464 [00:30<00:00, 15.31it/s]


In [ ]:
os.makedirs("./scidocs/eval", exist_ok=True)

args = Args(
    encoder='BAAI/bge-base-en-v1.5',
    fp16=True,
    add_instruction=False,
    k=100,
    corpus_data=eval_corpus_file,
    query_data=eval_queries_file,
    save_metrics=True,
    save_metrics_path="./scidocs/eval/",
    save_retrieval_results=True,
    save_retrieval_results_path="./scidocs/eval/",
    save_reranking_results=True,
    save_reranking_results_path="./scidocs/eval/",
)

main(args)

In [ ]:
# Evaluate using namespace-Pt/msmarco
# It is a big dataset.
# args = Args(
#     encoder='BAAI/bge-base-en-v1.5',
#     fp16=True,
#     add_instruction=False,
#     k=100,
#     save_metrics=False,
#     save_retrieval_results=False,
#     save_reranking_results=False,
# )

# main(args)

<b> Save evaluation result to Google Drive </b>

In [ ]:
from distutils.dir_util import copy_tree

drive_output_dir_eval = os.path.join(drive_output_dir, 'eval')

source_dir = "./scidocs"
dest_dir = drive_output_dir_eval

# Copy source directory to destination directory
copy_tree(source_dir, dest_dir)

['/content/drive/My Drive/Colab Notebooks/reranker/models/scidocs_3k_all_pos_20epoch/eval/eval_corpus.jsonl',
 '/content/drive/My Drive/Colab Notebooks/reranker/models/scidocs_3k_all_pos_20epoch/eval/eval/finetuned_reranking_results.json',
 '/content/drive/My Drive/Colab Notebooks/reranker/models/scidocs_3k_all_pos_20epoch/eval/eval/retrieval_results.json',
 '/content/drive/My Drive/Colab Notebooks/reranker/models/scidocs_3k_all_pos_20epoch/eval/eval/bge_reranking_results_sorted.json',
 '/content/drive/My Drive/Colab Notebooks/reranker/models/scidocs_3k_all_pos_20epoch/eval/eval/bge_reranking_results_metrics.json',
 '/content/drive/My Drive/Colab Notebooks/reranker/models/scidocs_3k_all_pos_20epoch/eval/eval/bge_reranking_results.json',
 '/content/drive/My Drive/Colab Notebooks/reranker/models/scidocs_3k_all_pos_20epoch/eval/eval/retrieval_results_metrics.json',
 '/content/drive/My Drive/Colab Notebooks/reranker/models/scidocs_3k_all_pos_20epoch/eval/eval/merged_reranking_results.json'

## TODO: evaluate using ragas

In [ ]:
from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
import tqdm
persist_directory = "./vectorstore/"
embedding = HuggingFaceEmbeddings(model_name="")